### Decimal to Binary Conversion

Single layer feedforward neural network that implements decimal to binary conversion.

In [1]:
%run core.ipynb

In [2]:
# %load dec2bin.py
import numpy as np


def int2dec(n):
    return np.array(tuple(0.99 if i == n else 0.01 for i in range(10)))


def dec2int(X):
    for i, x in enumerate(X):
        if x > 0.5:
            return i


def int2bin(n):
    b = []
    while n > 0:
        n, x = divmod(n, 2)
        b.append(x)
    return np.array(b + [0] * (4 - len(b)))


def bin2int(X):
    return sum(2**i if x > 0.5 else 0 for i, x in enumerate(X))


In [3]:
for _ in range(3):
    n = random.randint(0, 9)
    print(f"digit:		{_blu(n)}")
    print(f"decimal:	{int2dec(n)}")
    print(f"binary:		{int2bin(n)}")
    assert dec2int(int2dec(n)) == n
    assert bin2int(int2bin(n)) == n

digit:		0
decimal:	[0.99 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01]
binary:		[0 0 0 0]
digit:		3
decimal:	[0.01 0.01 0.01 0.99 0.01 0.01 0.01 0.01 0.01 0.01]
binary:		[1 1 0 0]
digit:		5
decimal:	[0.01 0.01 0.01 0.01 0.01 0.99 0.01 0.01 0.01 0.01]
binary:		[1 0 1 0]


#### Manual Weights and Biasis

In [4]:
def get_dec2bin_weights_and_biases():
    A = []
    for i in range(10):
        A.append(int2bin(i))
    W = np.transpose(np.array(A))
    B = - (np.sigmoid(0.99) - np.sigmoid(0.01)) / 2 * np.ones(4)
    return W, B


In [5]:
def run_dec2bin(W, B):
    for i in range(10):
        X = int2dec(i)
        Y = np.sigmoid(np.dot(W, X) + B)
        x, y = dec2int(X), bin2int(Y)
        Y = np.reshape(Y, (4,))
        assert x == y
        print(f'{x} -> {Y} = {[1 if t > 0.5 else 0 for t in Y]} {_ok}')


In [6]:
W, B = get_dec2bin_weights_and_biases()
run_dec2bin(W, B)

0 -> [0.48418179 0.48168471 0.48168471 0.47669341] = [0, 0, 0, 0] ✔
1 -> [0.71437046 0.48168471 0.48168471 0.47669341] = [1, 0, 0, 0] ✔
2 -> [0.48418179 0.71232564 0.48168471 0.47669341] = [0, 1, 0, 0] ✔
3 -> [0.71437046 0.71232564 0.48168471 0.47669341] = [1, 1, 0, 0] ✔
4 -> [0.48418179 0.48168471 0.71232564 0.47669341] = [0, 0, 1, 0] ✔
5 -> [0.71437046 0.48168471 0.71232564 0.47669341] = [1, 0, 1, 0] ✔
6 -> [0.48418179 0.71232564 0.71232564 0.47669341] = [0, 1, 1, 0] ✔
7 -> [0.71437046 0.71232564 0.71232564 0.47669341] = [1, 1, 1, 0] ✔
8 -> [0.48418179 0.48168471 0.48168471 0.70820995] = [0, 0, 0, 1] ✔
9 -> [0.71437046 0.48168471 0.48168471 0.70820995] = [1, 0, 0, 1] ✔


#### Weights and Biasis using Delta Rule

In [7]:
from delta_rule import delta_rule


def get_dec2bin_delta_rule_test_data():
    D = {n: int2dec(n) for n in range(10)}
    B = {n: int2bin(n) for n in range(10)}
    N = []
    for i in range(4):
        N.append([(D[n], B[n][i]) for n in range(10)])
    return N


def get_dec2bin_delta_rule_weights_and_biases(a, da, s=0.01, epochs=50):
    W, B = [], []
    for T in get_dec2bin_delta_rule_test_data():
        w, b = delta_rule(T, a=a, da=da, s=s, epochs=epochs)
        W.append(w)
        B.append(b)
    return np.array(W), np.array(B)


def run_dec2bin_delta_rule(W, B, a, classify):
    for n in range(10):
        X = tuple(int2bin(n))
        Y = tuple(a(x) for x in np.dot(W, int2dec(n)) + B)
        Z = tuple(classify(t) for t in Y)
        if X == Z:
            print(f"{n} =	{_rnd(Y, n=2)} =	{Z} {_ok}")
        else:
            print(_red(f"{n} ≠	{_rnd(Y, n=2)} =	{Z}	≠	{X} {_nok}"))


In [8]:
get_dec2bin_delta_rule_test_data()[0]

[(array([0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), 0),
 (array([0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), 1),
 (array([0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), 0),
 (array([0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), 1),
 (array([0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01]), 0),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01]), 1),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01]), 0),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01]), 1),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01]), 0),
 (array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99]), 1)]

In [9]:
a, da, classify = linear, dx_linear, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 150

W, B = get_dec2bin_delta_rule_weights_and_biases(a, da, s=s, epochs=epochs)
run_dec2bin_delta_rule(W, B, a, classify)

0 =	(0.02, 0.18, 0.04, 0.05) =	(0, 0, 0, 0) ✔
1 =	(0.9, 0.04, 0.07, -0.03) =	(1, 0, 0, 0) ✔
2 =	(0.09, 0.82, 0.18, 0.17) =	(0, 1, 0, 0) ✔
3 =	(0.87, 0.81, 0.01, -0.02) =	(1, 1, 0, 0) ✔
4 =	(0.22, 0.16, 0.77, 0.08) =	(0, 0, 1, 0) ✔
5 =	(0.85, 0.11, 0.93, 0.0) =	(1, 0, 1, 0) ✔
6 =	(0.18, 0.83, 0.92, 0.09) =	(0, 1, 1, 0) ✔
7 =	(0.92, 0.96, 0.81, 0.04) =	(1, 1, 1, 0) ✔
8 =	(0.18, 0.07, 0.09, 0.82) =	(0, 0, 0, 1) ✔
9 =	(0.79, 0.03, 0.17, 0.81) =	(1, 0, 0, 1) ✔


In [10]:
a, da, classify = relu, dx_relu, lambda x: heaviside(x, offset=0.4)
s, epochs = 0.01, 150

W, B = get_dec2bin_delta_rule_weights_and_biases(a, da, s=s, epochs=epochs)
run_dec2bin_delta_rule(W, B, a, classify)

0 =	(0.21, 0.06, 0.14, 0.0) =	(0, 0, 0, 0) ✔
1 =	(0.91, 0.05, 0.13, 0.0) =	(1, 0, 0, 0) ✔
2 =	(0.18, 0.9, 0.19, 0.01) =	(0, 1, 0, 0) ✔
3 =	(0.92, 0.9, 0.12, 0.09) =	(1, 1, 0, 0) ✔
4 =	(0.05, 0.0, 0.86, 0.0) =	(0, 0, 1, 0) ✔
5 =	(0.92, 0.0, 0.83, 0.05) =	(1, 0, 1, 0) ✔
6 =	(0.05, 0.8, 0.81, 0.07) =	(0, 1, 1, 0) ✔
7 =	(0.84, 0.94, 0.79, 0.07) =	(1, 1, 1, 0) ✔
8 =	(0.15, 0.19, 0.13, 0.89) =	(0, 0, 0, 1) ✔
9 =	(0.79, 0.15, 0.03, 0.83) =	(1, 0, 0, 1) ✔


In [11]:
a, da, classify = np.sigmoid, np.dx_sigmoid, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 1500

W, B = get_dec2bin_delta_rule_weights_and_biases(a, da, s=s, epochs=epochs)
run_dec2bin_delta_rule(W, B, a, classify)

0 =	(0.26, 0.24, 0.2, 0.19) =	(0, 0, 0, 0) ✔
1 =	(0.77, 0.19, 0.2, 0.15) =	(1, 0, 0, 0) ✔
2 =	(0.26, 0.73, 0.25, 0.13) =	(0, 1, 0, 0) ✔
3 =	(0.76, 0.71, 0.22, 0.15) =	(1, 1, 0, 0) ✔
4 =	(0.23, 0.22, 0.72, 0.18) =	(0, 0, 1, 0) ✔
5 =	(0.75, 0.22, 0.71, 0.16) =	(1, 0, 1, 0) ✔
6 =	(0.28, 0.72, 0.74, 0.13) =	(0, 1, 1, 0) ✔
7 =	(0.73, 0.74, 0.75, 0.19) =	(1, 1, 1, 0) ✔
8 =	(0.25, 0.22, 0.2, 0.67) =	(0, 0, 0, 1) ✔
9 =	(0.72, 0.21, 0.21, 0.64) =	(1, 0, 0, 1) ✔


#### Weights and Biasis using Perceptron

In [12]:
from perceptron import perceptron
get_dec2bin_perceptron_test_data = get_dec2bin_delta_rule_test_data


def get_dec2bin_perceptron_weights_and_biases(s=0.01, epochs=50):
    W, B = [], []
    for T in get_dec2bin_perceptron_test_data():
        w, b = perceptron(T, s=s, epochs=epochs)
        W.append(w)
        B.append(b)
    return np.array(W), np.array(B)


def run_dec2bin_perceptron(W, B):
    for n in range(10):
        X = tuple(int2bin(n))
        Y = tuple(np.dot(W, int2dec(n)) + B)
        Z = tuple(heaviside(t) for t in Y)
        if X == Z:
            print(f"{n} =	{_rnd(Y, n=3)} =	{Z} {_ok}")
        else:
            print(_red(f"{n} ≠	{_rnd(Y, n=3)} =	{Z}	≠	{X} {_nok}"))


In [13]:
s, epochs = 0.001, 6

W, B = get_dec2bin_perceptron_weights_and_biases(s=s, epochs=epochs)
run_dec2bin_perceptron(W, B)

0 =	(-0.001, -0.001, -0.001, -0.002) =	(0, 0, 0, 0) ✔
1 =	(0.001, -0.001, -0.001, -0.002) =	(1, 0, 0, 0) ✔
2 =	(-0.001, 0.002, -0.001, -0.002) =	(0, 1, 0, 0) ✔
3 =	(0.001, 0.002, -0.001, -0.002) =	(1, 1, 0, 0) ✔
4 =	(-0.001, -0.001, 0.002, -0.002) =	(0, 0, 1, 0) ✔
5 =	(0.001, -0.001, 0.002, -0.001) =	(1, 0, 1, 0) ✔
6 =	(-0.001, 0.002, 0.002, -0.001) =	(0, 1, 1, 0) ✔
7 =	(0.001, 0.0, 0.0, -0.001) =	(1, 1, 1, 0) ✔
8 =	(-0.001, -0.001, -0.001, 0.001) =	(0, 0, 0, 1) ✔
9 =	(0.001, -0.001, -0.001, 0.001) =	(1, 0, 0, 1) ✔
